In [32]:
# Tensorflow / Keras
from tensorflow import keras # for building Neural Networks
print('Tensorflow/Keras: %s' % keras.__version__) # print version
from keras.models import Sequential # for creating a linear stack of layers for our Neural Network
from keras import Input # for instantiating a keras tensor
from keras.layers import Masking,Bidirectional, LSTM, RepeatVector, Dense, TimeDistributed # for creating layers inside the Neural Network
from keras.optimizers import Adam
# Data manipulation
import pandas as pd # for data manipulation
print('pandas: %s' % pd.__version__) # print version
import numpy as np # for data manipulation
print('numpy: %s' % np.__version__) # print version

# Sklearn
import sklearn
print('sklearn: %s' % sklearn.__version__) # print version
from sklearn.preprocessing import MinMaxScaler # for feature scaling
from sklearn.model_selection import train_test_split

# Visualization
import plotly 
import plotly.express as px
import plotly.graph_objects as go
print('plotly: %s' % plotly.__version__) # print version

#file accessing
import os
# time stuff
from datetime import timedelta
import calendar


Tensorflow/Keras: 2.9.0
pandas: 1.4.2
numpy: 1.22.4
sklearn: 1.1.1
plotly: 5.9.0


In [33]:
#df=pd.read_csv('InSAR_data_south/displacement/export_dataframe.csv')
#df

In [34]:
#get displacement for one dot over the whole time 60 20 20
#open file by file
CSVpath="InSAR_data_south/displacement/CSV/test/"
dir_list=[]
lon_south=pd.read_csv('InSAR_data_south/longitude.csv')
lat_south=pd.read_csv('InSAR_data_south/latitude.csv')
for f in os.listdir(CSVpath):
    name, ext=os.path.splitext(f)
    if(ext=='.csv'):
        dir_list.append(f)

dir_list.sort()
print(dir_list)


['20141108.csv', '20141202.csv', '20141226.csv', '20150212.csv', '20150308.csv', '20150401.csv', '20150425.csv', '20150519.csv', '20150612.csv', '20150706.csv', '20150730.csv', '20150823.csv', '20150916.csv']


In [35]:
# df=pd.DataFrame()
# for f in dir_list:
#     name, ext=os.path.splitext(f)
#     if(ext=='.csv'):
#         print(CSVpath+f)
#         df_temp=pd.read_csv(CSVpath+f)
#         df_temp.columns=lon_south.columns
#         df_temp.index=lat_south.columns[:-1]
#         df_temp=df_temp.unstack().reset_index()
#         df_temp.columns=['Longitude','Latitude',name]
#         #print(df1.head())
#         #put it onto the main'
#         df1=pd.DataFrame(data=df_temp.iloc[:,2:3])
#         df1=df_temp.set_index([df_temp.columns[0],df_temp.columns[1]])
        
        
#         df=pd.concat([df,df1], axis=1)
df=pd.read_csv('InSAR_data_south/displacement/export_dataframe1.csv')



In [36]:
# Convert dates to year-months
#df['Year-Month']= (pd.to_datetime(df['Date'], yearfirst=True)).dt.strftime('%Y-%m')
df=df.set_index([df.columns[0],df.columns[1]])
df.columns=pd.to_datetime(df.columns, format='%Y%m%d')
timestep=10
#dftest


In [37]:
df
df=df.iloc[100000:100250]

In [38]:
#df.query("Longitude==-117.6414 and Latitude==37.6236")
# print(dftest.query('20141108'))
#df   

In [39]:
#tup=(-119.9588,35.185)
tup=(-119.2990,35.8852)
#tup=(-120.2888,34.8349)
#tup=(-117.6414,37.6236)
#print(dftest.loc[tup])

In [40]:

def shaping(datain, timestep):
    #print(arr)
    cnt=0
    
    for row in range(len(datain.index)):
    # Convert input dataframe to array and flatten
        arr=datain.iloc[row].to_numpy().flatten() # flatten row
        arr[np.isnan(arr)] = -1000
        for mth in range(0, len(datain.columns)-(2*timestep)+1): # Define range lenght of the dates - 2* amount of timesep?? +1
            cnt=cnt+1 # Gives us the number of samples. Later used to reshape the data
            X_start=mth # Start month for inputs of each sample
            X_end=mth+timestep # End month for inputs of each sample
            Y_start=mth+timestep # Start month for targets of each sample. Note, start is inclusive and end is exclusive, that's why X_end and Y_start is the same number
            Y_end=mth+2*timestep # End month for targets of each sample.  
            
            # Assemble input and target arrays containing all samples
            if cnt==1:
                X_comb=arr[X_start:X_end]
                Y_comb=arr[Y_start:Y_end]
            else: 
                X_comb=np.append(X_comb, arr[X_start:X_end])
                Y_comb=np.append(Y_comb, arr[Y_start:Y_end])
    
    # Reshape input and target arrays
    X_out=np.reshape(X_comb, (cnt, timestep, 1))
    Y_out=np.reshape(Y_comb, (cnt, timestep, 1))
    return X_out, Y_out

In [41]:
##### Step 2 - Prepare data
df=df.dropna(axis=0, how='all')
# # Split data into train and test dataframes
# df_train_temp=dftest.iloc[:, :88].copy()
# df_test_temp=dftest.iloc[:, 88:110].copy()
# #pick a location to test
# df_train = df_train_temp[df_train_temp.index==tup].copy() 
# df_test = df_test_temp[df_test_temp.index==tup].copy()

#print(df_test.columns)
#print(df_test)
#df_train.columns=['Date','Disp']
#df_test.columns=['Date','Disp']


# # Split data into train and test dataframes
# df_train=df2_pivot.iloc[:, 0:-2*timestep].copy()
# df_test=df2_pivot.iloc[:, -2*timestep:].copy()

# # Select one location
# dfloc_train = df_train[df_train.index==location].copy()
# dfloc_test = df_test[df_test.index==location].copy()
# #print(df_test.columns)
# Select one location
#dfloc_train = df_train[df_train.index==location].copy()
#dfloc_test = df_test[df_test.index==location].copy()


In [42]:
# Use previously defined shaping function to reshape the data for LSTM
# for x in range(len(df.index)):
#     X_train, Y_train += shaping(datain=df_train_temp.iloc[x], timestep= timestep)
#     X_test, Y_test += shaping(datain=df_test_temp.iloc[x], timestep=timestep)
train, test = train_test_split(df, test_size=0.2)
# Split data into samples


In [43]:

X_train, Y_train = shaping(datain=train, timestep= timestep)
X_test, Y_test = shaping(datain=test, timestep=timestep)
#print("Xtrain:", X_train.shape)

#print("Ytrain:", Y_train)

#print("Xtrain:", X_train.size)

In [44]:
##### Step 3 - Specify the structure of a Neural Network
model = Sequential(name="LSTM-Model") # Model
model.add(Masking(mask_value=-1000, input_shape=(timestep, 1)))
model.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='Input-Layer')) # Input Layer - need to speicfy the shape of inputs
model.add(LSTM(units=64, recurrent_activation='sigmoid', activation='relu',stateful=False, return_sequences=True))

model.add(LSTM(units=64, recurrent_activation='sigmoid', activation='relu',stateful=False, return_sequences=True))
#model.add(LSTM(units=50, activation='relu'), name='Hidden-LSTM-Encoder-Layer') # Encoder Layer
model.add(TimeDistributed(Dense(units=1, activation='linear'), name='Output-Layer')) # Output Layer, Linear(x) = x



In [45]:
##### Step 4 - Compile the model
#from keras.optimizers import adam_v2
#optimizer = adam_v2.Adam(learning_rate=1)
optimizer=Adam( 0.001)

model.compile(optimizer='adam', # default='rmsprop', an algorithm to be used in backpropagation
              loss='mean_squared_error', # Loss function to be optimized. A string (name of loss function), or a tf.keras.losses.Loss instance.
              metrics=['MeanSquaredError', 'MeanAbsoluteError'], # List of metrics to be evaluated by the model during training and testing. Each of this can be a string (name of a built-in function), function or a tf.keras.metrics.Metric instance. 
              loss_weights=None, # default=None, Optional list or dictionary specifying scalar coefficients (Python floats) to weight the loss contributions of different model outputs.
              weighted_metrics=None, # default=None, List of metrics to be evaluated and weighted by sample_weight or class_weight during training and testing.
              run_eagerly=None, # Defaults to False. If True, this Model's logic will not be wrapped in a tf.function. Recommended to leave this as None unless your Model cannot be run inside a tf.function.
              steps_per_execution=None # Defaults to 1. The number of batches to run during each tf.function call. Running multiple batches inside a single tf.function call can greatly improve performance on TPUs or small models with a large Python overhead.
        
             )

In [46]:


##### Step 5 - Fit the model on the dataset
history = model.fit(X_train, # input data
                    Y_train, # target data
                    batch_size=3, # Number of samples per gradient update. If unspecified, batch_size will default to 32.
                    epochs=1000, # default=1, Number of epochs to train the model. An epoch is an iteration over the entire x and y data provided
                    verbose=2, # default='auto', ('auto', 0, 1, or 2). Verbosity mode. 0 = silent, 1 = progress bar, 2 = one line per epoch. 'auto' defaults to 1 for most cases, but 2 when used with ParameterServerStrategy.
                    callbacks=None, # default=None, list of callbacks to apply during training. See tf.keras.callbacks
                    validation_split=0.2, # default=0.0, Fraction of the training data to be used as validation data. The model will set apart this fraction of the training data, will not train on it, and will evaluate the loss and any model metrics on this data at the end of each epoch. 
                    #validation_data=(X_test, y_test), # default=None, Data on which to evaluate the loss and any model metrics at the end of each epoch. 
                    shuffle=True, # default=True, Boolean (whether to shuffle the training data before each epoch) or str (for 'batch').
                    class_weight=None, # default=None, Optional dictionary mapping class indices (integers) to a weight (float) value, used for weighting the loss function (during training only). This can be useful to tell the model to "pay more attention" to samples from an under-represented class.
                    sample_weight=None, # default=None, Optional Numpy array of weights for the training samples, used for weighting the loss function (during training only).
                    initial_epoch=0, # Integer, default=0, Epoch at which to start training (useful for resuming a previous training run).
                    steps_per_epoch=None, # Integer or None, default=None, Total number of steps (batches of samples) before declaring one epoch finished and starting the next epoch. When training with input tensors such as TensorFlow data tensors, the default None is equal to the number of samples in your dataset divided by the batch size, or 1 if that cannot be determined. 
                    validation_steps=None, # Only relevant if validation_data is provided and is a tf.data dataset. Total number of steps (batches of samples) to draw before stopping when performing validation at the end of every epoch.
                    validation_batch_size=None, # Integer or None, default=None, Number of samples per validation batch. If unspecified, will default to batch_size.
                    validation_freq=100, # default=1, Only relevant if validation data is provided. If an integer, specifies how many training epochs to run before a new validation run is performed, e.g. validation_freq=2 runs validation every 2 epochs.
                    max_queue_size=10, # default=10, Used for generator or keras.utils.Sequence input only. Maximum size for the generator queue. If unspecified, max_queue_size will default to 10.
                    workers=1, # default=1, Used for generator or keras.utils.Sequence input only. Maximum number of processes to spin up when using process-based threading. If unspecified, workers will default to 1.
                    use_multiprocessing=True, # default=False, Used for generator or keras.utils.Sequence input only. If True, use process-based threading. If unspecified, use_multiprocessing will default to False. 
                
                   )



Epoch 1/1000
364/364 - 12s - loss: 15206.0537 - mean_squared_error: 15403.5342 - mean_absolute_error: 25.7456 - 12s/epoch - 32ms/step
Epoch 2/1000
364/364 - 4s - loss: 14984.3740 - mean_squared_error: 15178.9756 - mean_absolute_error: 28.0222 - 4s/epoch - 10ms/step
Epoch 3/1000
364/364 - 4s - loss: 14806.9199 - mean_squared_error: 14999.2129 - mean_absolute_error: 29.6480 - 4s/epoch - 10ms/step
Epoch 4/1000
364/364 - 4s - loss: 14614.0908 - mean_squared_error: 14803.8838 - mean_absolute_error: 29.9245 - 4s/epoch - 10ms/step
Epoch 5/1000
364/364 - 3s - loss: 14051.6123 - mean_squared_error: 14234.1006 - mean_absolute_error: 31.0031 - 3s/epoch - 10ms/step
Epoch 6/1000
364/364 - 4s - loss: 12924.7246 - mean_squared_error: 13092.5771 - mean_absolute_error: 28.5956 - 4s/epoch - 10ms/step
Epoch 7/1000
364/364 - 4s - loss: 12195.6230 - mean_squared_error: 12354.0059 - mean_absolute_error: 27.9829 - 4s/epoch - 10ms/step
Epoch 8/1000
364/364 - 3s - loss: 11466.5088 - mean_squared_error: 11615.4

In [47]:
##### Step 6 - Use model to make predictions
# Predict results on training data
pred_train = model.predict(X_train)
# Predict esults on test data
pred_test = model.predict(X_test)

12/12 [==============================] - 0s 8ms/step


In [48]:
##### Step 7 - Print Performance Summary
print("")
print('-------------------- Model Summary --------------------')
model.summary() # print model summary
print("")
print('-------------------- Weights and Biases --------------------')
print("Too many parameters to print but you can use the code provided if needed")
print("")
#for layer in model.layers:
#    print(layer.name)
#    for item in layer.get_weights():
#        print("  ", item)
#print("")

# Print the last value in the evaluation metrics contained within history file
print('-------------------- Evaluation on Training Data --------------------')
for item in history.history:
    print("Final", item, ":", history.history[item][-1])
print("")

# Evaluate the model on the test data using "evaluate"
print('-------------------- Evaluation on Test Data --------------------')
results = model.evaluate(X_test, Y_test)
print("")


-------------------- Model Summary --------------------
Model: "LSTM-Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking_2 (Masking)         (None, 10, 1)             0         
                                                                 
 Input-Layer (InputLayer)    multiple                  0         
                                                                 
 lstm_4 (LSTM)               (None, 10, 64)            16896     
                                                                 
 lstm_5 (LSTM)               (None, 10, 64)            33024     
                                                                 
 Output-Layer (TimeDistribut  (None, 10, 1)            65        
 ed)                                                             
                                                                 
Total params: 49,985
Trainable params: 49,985
Non-trainable param

In [49]:
# Plot average monthly temperatures (actual and predicted) for test (out of time) data
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.array(df_train.columns),
                         y=np.array(df_train.values).flatten(),
                         mode='lines',
                         name='Subsidence - Actual (Train)',
                         opacity=0.8,
                         line=dict(color='blue', width=1)
                        ))
#Trace for actual temperatures
fig.add_trace(go.Scatter(x=np.array(df_test.columns),
                         y=np.array(df_test.values).flatten(),
                         mode='lines',
                         name='Subsidence - Actual (Test)',
                         opacity=0.8,
                         line=dict(color='black', width=1)
                        ))

# Trace for predicted temperatures
fig.add_trace(go.Scatter(x=np.array(df_test.columns[-timestep:]),
                         y=pred_test.flatten(),
                         mode='lines',
                         name='Subsidence - Predicted (Test)',
                         opacity=0.8,
                         line=dict(color='red', width=1)
                        ))

# Change chart background color
fig.update_layout(dict(plot_bgcolor = 'white'))

# Update axes lines
fig.update_xaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Month-Year'
                )

fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='lightgrey', 
                 zeroline=True, zerolinewidth=1, zerolinecolor='lightgrey', 
                 showline=True, linewidth=1, linecolor='black',
                 title='Subsidence'
                )

# Set figure title
fig.update_layout(title=dict(text="Subsidence", font=dict(color='black')),
                  legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1)
                 )
fig.show()

NameError: name 'df_train' is not defined